In [ ]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import pandas as pd
import re
import os
# it is used to covert my text to speech pyttsx3
import pyttsx3
#recognise speech  word
import speech_recognition as sr

In [ ]:
#below we create a object for voice taken from pyttsx3
engine = pyttsx3.init()
#get voice from module
voices = engine.getProperty('voices')
#there are so many voice we choose a location of voice which want to use through its index with its id
# print(voices[1].id)
engine.setProperty('voice', voices[1].id)
#set the frequency of  the sound how much you want
engine.setProperty('rate', 150)

def speak(audio):
    #say listen until you can't reply
    engine.say(audio)
    engine.runAndWait()

In [ ]:
def takeCommand():
    #It takes microphone input from the user and returns string output
    #here below a we create object to recognize audio of human 
    r = sr.Recognizer()
    #get audio  from the microphone 
    with sr.Microphone() as source:
        #adjust ambient noise is for removing background  voice 
        audio = r.adjust_for_ambient_noise(source)
        print("Listening...")
        #hear until 1min if u can't respond
        r.pause_threshold = 1
        #it will listen u
        audio = r.listen(source)

    try:
        print("Recognizing...")   
        #recognise  your language through google english-indian
        query = r.recognize_google(audio, language='en-in')
        print(f"User said: {query}\n")

    except Exception as e:
        # print(e)    
        print("Say that again please...")  
        return "None"
    return query

In [ ]:
# basepath = "chats//"
# human = []
# bot = []

# for files in os.listdir(basepath):
#     data = open(os.path.join(basepath,files),'r').readlines()
#     for i in data:
#         if '- - ' in i:
#             human.append(i)
# #             print('human: ',i)
#         else:
# #             print('bot',i)
#             bot.append(i)
    


In [ ]:
stop_words = stopwords.words('english')
def clean_data(text):
    #it is used to replace this term re.sub(pattern,replace,string)
    text = re.sub('<[^>]*>', '', text)
    #it is used to find all the matches and remove emoji
    emoticons = re.findall('(?::|;|=)(?:-)',text)
    #replace punctuation 
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    rm_words = [w for w in text.split() if w.lower() not in stop_words]
    return ' '.join(rm_words)
    return text

In [ ]:
chats = pd.read_csv('rdany_conversations_2016-03-01.csv')
chats = chats[['source','text']]
chats['train_text'] = chats['text'].apply(clean_data)

human = []
bot = []
for i in range(len(chats)):
    #here we are checking human and bot if it not similiar we will continue
    if chats.iloc[i,0]==chats.iloc[i-1,0]:
        continue
    if chats.iloc[i,0] == 'human':
        human.append(chats.iloc[i,-1])
    else:
        bot.append(chats.iloc[i,1])

In [ ]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.

def LemTokens(tokens):
    #split the text and store in list
    return [lemmer.lemmatize(token) for token in tokens]
#we are creating a dictionary of punctuation
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    #call above function and parameter in word tokenize  and lower text value
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
def response(user_response):

    user_response = [user_response]
    robo_response=''
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(human)
    tfidf_user_response = TfidfVec.transform(user_response)
    vals = cosine_similarity(tfidf_user_response, tfidf)
    #this will give you the index value
    idx=vals.argsort()[0][-1]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-1]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return (robo_response)
    else:
        robo_response = robo_response+bot[idx]
        return (robo_response)

In [ ]:
flag=True
print("ROBO: My name is Robo. I will answer your queries. If you want to exit, type Bye!")
speak("My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = takeCommand()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            print("ROBO: ",end="")
            print(response(user_response))
            speak(response(user_response))
    else:
        flag=False
        print("ROBO: Bye! take care..")
        speak("Bye! take care..")